In [1]:
!pip install soundfile

In [2]:
import numpy as np
import soundfile as sf
from IPython.display import Audio

In [3]:
#Sampling rate and period
Fs = 44100
Ts = 1 / Fs

# Op-amp stage component values
C1 = 10e-9
R1 = Ts / (2 * C1)

C2 = 47e-9
R2 = Ts / (2 * C2)

R3 = 10e3
R4 = 1e6
R5 = 4.7e3
R8 = 1e6

# Distortion knob
pot = 0.75
k = 8
R6 = (np.exp(-k * pot) - np.exp(-k)) / (1 - np.exp(-k)) * 1000000
Rn = R5 + R6

# G-values
Ga = 1 + (R3 / R1)
Gb = 1 + (R2 / Rn)
Gh = 1 + (R4 / (Gb * Rn))
Gx = (1 / R8) + (1 / (Ga * R1))

# Coefficients
b0 = Gh / (Ga * R1 * Gx)
b1 = ((R3 / (Ga * R1)) - 1) * (Gh / Gx)
b2 = (-R2 * R4) / (Gb * Rn)

# input signal: wave file
input_signal, Fs = sf.read('testaudio.wav')
N = len(input_signal)

# initialise the output signal
y = np.zeros(N)

# initial state values
x1 = 0
x2 = 0

# signal processing for Op-amp stage
for n in range(N):
    Vin = input_signal[n]
    Vout = b0 * Vin + b1 * x1 + b2 * x2

    Vx = (1 / Gh) * Vout + ((R2 * R4) / (Gb * Gh * Rn)) * x2
    VR1 = (Vin - Vx + (R3 * x1)) / Ga

    VRn = (Vx - (R2 * x2)) / Gb
    VR2 = (R2 / Rn) * VRn + (R2 * x2)

    x1 = (2 / R1) * VR1 - x1
    x2 = (2 / R2) * VR2 - x2

    y[n] = Vout


# Clipping stage

# Germanium diode params
Is = 100e-9  # saturation current
Vt = 0.026  # thermal voltage
eta = 2  # emission coefficient

# clipping stage's component values
Rb = 10000
Ca = 1e-9
Ra = Ts / (2 * Ca)

# Output knob: 0 - 0.999
outputpot = 0.75
Re = 10000 * (1 - np.log10(1 + 9 * (1 - outputpot)))
Rd = 10000 - Re

Gg = (1 / Rd) + (1 / Re)

Vd = 0 # initial guess of Vd
Vout2 = Vd / (Rd * Gg) # initial Vout2

TOL = 1e-10 # a very smlall value close enough to zero
xa = 0 # initial state value for DK substitution circuit of a capacitor

z = np.zeros(N)

# clipping stage's sample-by-sample processing
for n in range(N):
    Vin2 = y[n] # discrete input signal

    fVd = 2 * Is * np.sinh(Vd / (eta * Vt)) + (Vd / Rb) + (-Vin2 / Rb) + (Vd / Ra) - xa + (Vd / Rd) + (-Vout2 / Rd)
    count = 0

    # a nested loop to find the Vd for each input sample
    while abs(fVd) > TOL and count < 10:
        der = ((2 * Is / (eta * Vt)) * np.cosh(Vd / (eta * Vt))) + (1 / Ra) + (1 / Rb) + (1 / Rd)
        Vd -= fVd / der #N-R update equation
        fVd = 2 * Is * np.sinh(Vd / (eta * Vt)) + (Vd / Rb) + (-Vin2 / Rb) + (Vd / Ra) - xa + (Vd / Rd) + (-Vout2 / Rd) #find Vd with updated Vd val.
        count += 1

    Vout2 = Vd / (Rd * Gg)
    xa = ((2 / Ra) * Vd) - xa # state update equation

    # z = final output vector
    z[n] = Vout2


In [4]:
sf.write('input.wav', input_signal, Fs)
sf.write('output.wav', z, Fs)

In [ ]:
# play the input (raw) signal
print("Input signal:")
Audio('input.wav', autoplay=False)

In [ ]:
# play the output signal
print("Processed output signal:")
Audio('output.wav', autoplay=False)